# Step 0: Cài đặt môi trường & Cấu hình API Keys

In [1]:
!pip install google-adk -q
!pip install litellm -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.1/218.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.7/335.7 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not cu

In [2]:
import os
import asyncio
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types

import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.ERROR)

In [3]:
os.environ["GOOGLE_API_KEY"] = ''

os.environ['OPENAI_API_KEY'] = ''

os.environ['ANTHROPIC_API_KEY'] = ''

os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"

In [4]:
MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash"

MODEL_GPT_4O = "openai/gpt-4.1"

MODEL_CLAUDE_SONNET = "anthropic/claude-sonnet-4-20250514"

# Step 1: Tra cứu thời tiết cơ bản

## Định nghĩa hàm get_weather

In [5]:
def get_weather(city: str) -> dict:
    """Lấy báo cáo thời tiết hiện tại cho một thành phố cụ thể.

    Tham số:
        city (str): Tên thành phố (ví dụ: "New York", "London", "Tokyo").

    Trả về:
        dict: Một từ điển chứa thông tin thời tiết.
              Bao gồm khóa 'status' ('success' hoặc 'error').
              Nếu 'success', có thêm khóa 'report' với chi tiết thời tiết.
              Nếu 'error', có khóa 'error_message' chứa thông báo lỗi.
    """
    print(f"--- Tool: get_weather được gọi cho thành phố: {city} ---")  # Ghi log khi tool được gọi
    city_normalized = city.lower().replace(" ", "")  # Chuẩn hóa tên thành phố đơn giản

    # Dữ liệu thời tiết giả lập
    mock_weather_db = {
        "newyork": {"status": "success", "report": "Thời tiết ở New York nắng, nhiệt độ 25°C."},
        "london": {"status": "success", "report": "Ở London trời nhiều mây, nhiệt độ 15°C."},
        "tokyo": {"status": "success", "report": "Tokyo đang mưa nhẹ, nhiệt độ 18°C."},
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return {"status": "error", "error_message": f"Xin lỗi, tôi không có thông tin thời tiết cho '{city}'."}

# Ví dụ sử dụng tool (kiểm tra)
print(get_weather("New York"))
print(get_weather("Paris"))

--- Tool: get_weather được gọi cho thành phố: New York ---
{'status': 'success', 'report': 'Thời tiết ở New York nắng, nhiệt độ 25°C.'}
--- Tool: get_weather được gọi cho thành phố: Paris ---
{'status': 'error', 'error_message': "Xin lỗi, tôi không có thông tin thời tiết cho 'Paris'."}


## Định nghĩa Agent

In [6]:
AGENT_MODEL = MODEL_GEMINI_2_0_FLASH

weather_agent = Agent(
    name="weather_agent_v1",
    model=AGENT_MODEL,
    description="Cung cấp thông tin thời tiết cho các thành phố cụ thể.",
    instruction=(
        "Bạn là một trợ lý thời tiết hữu ích. "
        "Khi người dùng hỏi về thời tiết ở một thành phố cụ thể, "
        "hãy sử dụng công cụ 'get_weather' để tìm thông tin. "
        "Nếu công cụ trả về lỗi, hãy thông báo cho người dùng một cách lịch sự. "
        "Nếu công cụ hoạt động thành công, hãy trình bày báo cáo thời tiết một cách rõ ràng."
    ),
    tools=[get_weather],
)

print(f"Agent '{weather_agent.name}' đã được tạo với model '{AGENT_MODEL}'.")


Agent 'weather_agent_v1' đã được tạo với model 'gemini-2.0-flash'.


## Thiết lập Runner và Session Service

In [7]:
session_service = InMemorySessionService()

APP_NAME = "weather_tutorial_app"
USER_ID = "user_1"
SESSION_ID = "session_001"

session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print(f"Đã tạo session: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

runner = Runner(
    agent=weather_agent,
    app_name=APP_NAME,
    session_service=session_service
)
print(f"Đã tạo Runner cho agent '{runner.agent.name}'.")


Đã tạo session: App='weather_tutorial_app', User='user_1', Session='session_001'
Đã tạo Runner cho agent 'weather_agent_v1'.


## Tương tác với Agent



In [8]:
from google.genai import types

async def call_agent_async(query: str, runner, user_id, session_id):
    """Gửi câu hỏi đến agent và in ra phản hồi cuối cùng."""
    print(f"\n>>> Câu hỏi từ người dùng: {query}")

    content = types.Content(role='user', parts=[types.Part(text=query)])

    final_response_text = "Agent không tạo ra phản hồi cuối cùng."

    async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
        print(f"  [Event] Tác giả: {event.author}, Loại: {type(event).__name__}, Cuối cùng: {event.is_final_response()}, Nội dung: {event.content}")

        if event.is_final_response():
            if event.content and event.content.parts:
                final_response_text = event.content.parts[0].text
            elif event.actions and event.actions.escalate:
                final_response_text = f"Agent đã gặp lỗi: {event.error_message or 'Không có thông báo cụ thể.'}"
            break

    print(f"<<< Phản hồi từ Agent: {final_response_text}")

## Chạy cuộc trò chuyện

In [9]:
async def run_conversation():
    await call_agent_async("Thời tiết ở London như thế nào?",
                           runner=runner,
                           user_id=USER_ID,
                           session_id=SESSION_ID)

    await call_agent_async("Còn ở Paris thì sao?",
                           runner=runner,
                           user_id=USER_ID,
                           session_id=SESSION_ID)  # Kỳ vọng nhận thông báo lỗi từ tool

    await call_agent_async("Cho tôi biết thời tiết ở New York",
                           runner=runner,
                           user_id=USER_ID,
                           session_id=SESSION_ID)

await run_conversation()


>>> Câu hỏi từ người dùng: Thời tiết ở London như thế nào?


  [Event] Tác giả: weather_agent_v1, Loại: Event, Cuối cùng: False, Nội dung: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=FunctionCall(id='adk-2bdd195d-b103-48ff-a986-dec4ee21ad7f', args={'city': 'London'}, name='get_weather'), function_response=None, text=None)] role='model'
--- Tool: get_weather được gọi cho thành phố: London ---
  [Event] Tác giả: weather_agent_v1, Loại: Event, Cuối cùng: False, Nội dung: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=FunctionResponse(will_continue=None, scheduling=None, id='adk-2bdd195d-b103-48ff-a986-dec4ee21ad7f', name='get_weather', response={'status': 'success', 'report': 'Ở London trời nhiều mây, nhiệt độ 15°C.'}), text=None)] role='user'
  [Event] Tác giả: weather_agent_v1, Lo

  [Event] Tác giả: weather_agent_v1, Loại: Event, Cuối cùng: False, Nội dung: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=FunctionCall(id='adk-4690394f-bf56-4111-bfcc-0750ff16e9be', args={'city': 'Paris'}, name='get_weather'), function_response=None, text=None)] role='model'
--- Tool: get_weather được gọi cho thành phố: Paris ---
  [Event] Tác giả: weather_agent_v1, Loại: Event, Cuối cùng: False, Nội dung: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=FunctionResponse(will_continue=None, scheduling=None, id='adk-4690394f-bf56-4111-bfcc-0750ff16e9be', name='get_weather', response={'status': 'error', 'error_message': "Xin lỗi, tôi không có thông tin thời tiết cho 'Paris'."}), text=None)] role='user'
  [Event] Tác giả: we

  [Event] Tác giả: weather_agent_v1, Loại: Event, Cuối cùng: False, Nội dung: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=FunctionCall(id='adk-b559b1f2-fd31-4fd6-a96a-e8337af6220e', args={'city': 'New York'}, name='get_weather'), function_response=None, text=None)] role='model'
--- Tool: get_weather được gọi cho thành phố: New York ---
  [Event] Tác giả: weather_agent_v1, Loại: Event, Cuối cùng: False, Nội dung: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=FunctionResponse(will_continue=None, scheduling=None, id='adk-b559b1f2-fd31-4fd6-a96a-e8337af6220e', name='get_weather', response={'status': 'success', 'report': 'Thời tiết ở New York nắng, nhiệt độ 25°C.'}), text=None)] role='user'
  [Event] Tác giả: weather_agent_

# Step 2: Sử dụng nhiều mô hình với LiteLLM

## Tạo và kiểm tra agent sử dụng GPT-4o của OpenAI

In [10]:
# --- Agent dùng GPT-4o ---
weather_agent_gpt = None  # Khởi tạo biến agent
runner_gpt = None         # Khởi tạo biến runner

try:
    weather_agent_gpt = Agent(
        name="weather_agent_gpt",
        # Điểm khác biệt: Dùng model qua wrapper LiteLLM
        model=LiteLlm(model=MODEL_GPT_4O),
        description="Cung cấp thông tin thời tiết (sử dụng GPT-4o).",
        instruction="Bạn là một trợ lý thời tiết thân thiện, được hỗ trợ bởi GPT-4o. "
                    "Hãy dùng công cụ 'get_weather' để xử lý các yêu cầu liên quan đến thời tiết theo thành phố. "
                    "Hiển thị kết quả rõ ràng nếu thành công hoặc trả lời lịch sự nếu có lỗi từ công cụ.",
        tools=[get_weather],  # Dùng lại công cụ đã định nghĩa
    )
    print(f"Đã tạo agent '{weather_agent_gpt.name}' với mô hình '{MODEL_GPT_4O}'.")

    # InMemorySessionService là dịch vụ lưu trữ đơn giản, không bền vững – phù hợp để demo.
    session_service_gpt = InMemorySessionService()  # Tạo riêng service cho agent này

    # Xác định thông tin định danh cho ngữ cảnh tương tác
    APP_NAME_GPT = "weather_tutorial_app_gpt"
    USER_ID_GPT = "user_1_gpt"
    SESSION_ID_GPT = "session_001_gpt"

    # Tạo session cụ thể cho cuộc trò chuyện này
    session_gpt = await session_service_gpt.create_session(
        app_name=APP_NAME_GPT,
        user_id=USER_ID_GPT,
        session_id=SESSION_ID_GPT
    )
    print(f"Đã tạo session: App='{APP_NAME_GPT}', User='{USER_ID_GPT}', Session='{SESSION_ID_GPT}'")

    # Tạo runner cho agent này
    runner_gpt = Runner(
        agent=weather_agent_gpt,
        app_name=APP_NAME_GPT,
        session_service=session_service_gpt
    )
    print(f"Đã tạo runner cho agent '{runner_gpt.agent.name}'.")

    # --- Kiểm thử agent GPT ---
    print("\n--- Đang kiểm tra agent GPT ---")
    await call_agent_async(query="Thời tiết ở Tokyo như thế nào?",
                           runner=runner_gpt,
                           user_id=USER_ID_GPT,
                           session_id=SESSION_ID_GPT)

except Exception as e:
    print(f"❌ Không thể tạo hoặc chạy agent GPT '{MODEL_GPT_4O}'. Kiểm tra lại API Key và tên model. Lỗi: {e}")

Đã tạo agent 'weather_agent_gpt' với mô hình 'openai/gpt-4.1'.
Đã tạo session: App='weather_tutorial_app_gpt', User='user_1_gpt', Session='session_001_gpt'
Đã tạo runner cho agent 'weather_agent_gpt'.

--- Đang kiểm tra agent GPT ---

>>> Câu hỏi từ người dùng: Thời tiết ở Tokyo như thế nào?
  [Event] Tác giả: weather_agent_gpt, Loại: Event, Cuối cùng: False, Nội dung: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=FunctionCall(id='call_R2nQi9m4WcQGaq7MG0r76QcL', args={'city': 'Tokyo'}, name='get_weather'), function_response=None, text=None)] role='model'
--- Tool: get_weather được gọi cho thành phố: Tokyo ---
  [Event] Tác giả: weather_agent_gpt, Loại: Event, Cuối cùng: False, Nội dung: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, functi

## Tạo và kiểm tra agent sử dụng mô hình Claude Sonnet của Anthropicss

In [11]:
# --- Agent sử dụng Claude Sonnet ---
weather_agent_claude = None  # Khởi tạo giá trị None
runner_claude = None         # Khởi tạo runner

try:
    weather_agent_claude = Agent(
        name="weather_agent_claude",
        # Điểm khác biệt: sử dụng LiteLLM để wrap model Claude
        model=LiteLlm(model=MODEL_CLAUDE_SONNET),
        description="Cung cấp thông tin thời tiết (sử dụng Claude Sonnet).",
        instruction="Bạn là một trợ lý thời tiết thân thiện, được hỗ trợ bởi Claude Sonnet. "
                    "Sử dụng công cụ 'get_weather' để xử lý yêu cầu thời tiết theo thành phố. "
                    "Phân tích output dạng từ điển từ tool (gồm 'status', 'report' hoặc 'error_message'). "
                    "Trình bày kết quả nếu thành công, hoặc báo lỗi một cách lịch sự nếu có.",
        tools=[get_weather],  # Dùng lại tool đã định nghĩa
    )
    print(f"Đã tạo agent '{weather_agent_claude.name}' với model '{MODEL_CLAUDE_SONNET}'.")

    # InMemorySessionService là dạng lưu trữ đơn giản, không bền vững – phù hợp để demo.
    session_service_claude = InMemorySessionService()  # Tạo riêng cho agent Claude

    # Xác định thông tin định danh cho ngữ cảnh tương tác
    APP_NAME_CLAUDE = "weather_tutorial_app_claude"
    USER_ID_CLAUDE = "user_1_claude"
    SESSION_ID_CLAUDE = "session_001_claude"

    # Tạo session cho cuộc trò chuyện này
    session_claude = await session_service_claude.create_session(
        app_name=APP_NAME_CLAUDE,
        user_id=USER_ID_CLAUDE,
        session_id=SESSION_ID_CLAUDE
    )
    print(f"Đã tạo session: App='{APP_NAME_CLAUDE}', User='{USER_ID_CLAUDE}', Session='{SESSION_ID_CLAUDE}'")

    # Tạo runner cho agent này
    runner_claude = Runner(
        agent=weather_agent_claude,
        app_name=APP_NAME_CLAUDE,
        session_service=session_service_claude
    )
    print(f"Đã tạo runner cho agent '{runner_claude.agent.name}'.")

    # --- Kiểm thử Agent Claude ---
    print("\n--- Đang kiểm tra agent Claude ---")
    await call_agent_async(query="Weather in London please.",
                           runner=runner_claude,
                           user_id=USER_ID_CLAUDE,
                           session_id=SESSION_ID_CLAUDE)

except Exception as e:
    print(f"❌ Không thể tạo hoặc chạy Claude agent '{MODEL_CLAUDE_SONNET}'. Kiểm tra lại API Key và tên model. Lỗi: {e}")

Đã tạo agent 'weather_agent_claude' với model 'anthropic/claude-sonnet-4-20250514'.
Đã tạo session: App='weather_tutorial_app_claude', User='user_1_claude', Session='session_001_claude'
Đã tạo runner cho agent 'weather_agent_claude'.

--- Đang kiểm tra agent Claude ---

>>> Câu hỏi từ người dùng: Weather in London please.

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

❌ Không thể tạo hoặc chạy Claude agent 'anthropic/claude-sonnet-4-20250514'. Kiểm tra lại API Key và tên model. Lỗi: litellm.BadRequestError: AnthropicException - {"type":"error","error":{"type":"invalid_request_error","message":"Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits."}}


# Step 3: Xây dựng một nhóm Agent - Phân quyền cho Lời chào và Tạm biệt

## Định nghĩa các Tool cho các Agent phụ

In [14]:
from typing import Optional


def say_hello(name: Optional[str] = None) -> str:
    """Trả về một lời chào đơn giản. Nếu có tên, lời chào sẽ bao gồm tên.

    Args:
        name (str, optional): Tên người cần chào. Mặc định sẽ dùng lời chào chung nếu không truyền tên.

    Returns:
        str: Câu chào thân thiện.
    """
    if name:
        greeting = f"Xin chào, {name}!"
        print(f"--- Tool: say_hello được gọi với tên: {name} ---")
    else:
        greeting = "Xin chào!"
        print(f"--- Tool: say_hello được gọi mà không có tên cụ thể (giá trị name: {name}) ---")
    return greeting

def say_goodbye() -> str:
    """Trả về một câu chào tạm biệt đơn giản để kết thúc cuộc hội thoại."""
    print(f"--- Tool: say_goodbye được gọi ---")
    return "Tạm biệt! Chúc bạn một ngày tốt lành."


In [15]:
print(say_hello("Alice"))
print(say_hello())
print(say_hello(name=None))

--- Tool: say_hello được gọi với tên: Alice ---
Xin chào, Alice!
--- Tool: say_hello được gọi mà không có tên cụ thể (giá trị name: None) ---
Xin chào!
--- Tool: say_hello được gọi mà không có tên cụ thể (giá trị name: None) ---
Xin chào!


## Định nghĩa các Sub-Agent (Agent Chào hỏi & Tạm biệt)

### Định nghĩa Greeting Agent (Agent Chào hỏi)

In [16]:
greeting_agent = None
try:
    greeting_agent = Agent(
        model = MODEL_GEMINI_2_0_FLASH,  # Dùng model đơn giản vì nhiệm vụ nhẹ
        name="greeting_agent",
        instruction="Bạn là Greeting Agent. NHIỆM VỤ DUY NHẤT của bạn là gửi lời chào thân thiện đến người dùng. "
                    "Hãy dùng công cụ 'say_hello' để tạo lời chào. "
                    "Nếu người dùng cung cấp tên, hãy truyền tên đó cho công cụ. "
                    "Không thực hiện bất kỳ tác vụ nào khác.",
        description="Xử lý lời chào đơn giản và chào hỏi bằng công cụ 'say_hello'.", # Cực kỳ quan trọng cho việc phân quyền
        tools=[say_hello],
    )
    print(f"✅ Agent '{greeting_agent.name}' đã được tạo với model '{greeting_agent.model}'.")
except Exception as e:
    print(f"❌ Không thể tạo Greeting Agent. Kiểm tra API Key ({greeting_agent.model}). Lỗi: {e}")

✅ Agent 'greeting_agent' đã được tạo với model 'gemini-2.0-flash'.


###  Định nghĩa Farewell Agent (Agent Tạm biệt)

In [17]:
farewell_agent = None
try:
    farewell_agent = Agent(
        model = MODEL_GEMINI_2_0_FLASH,
        name="farewell_agent",
        instruction="Bạn là Farewell Agent. NHIỆM VỤ DUY NHẤT của bạn là gửi lời tạm biệt lịch sự. "
                    "Hãy dùng công cụ 'say_goodbye' khi người dùng thể hiện ý định rời đi hoặc kết thúc cuộc hội thoại "
                    "(ví dụ như dùng các từ: 'bye', 'goodbye', 'cảm ơn tạm biệt', 'hẹn gặp lại'). "
                    "Không thực hiện bất kỳ hành động nào khác.",
        description="Xử lý lời tạm biệt đơn giản bằng công cụ 'say_goodbye'.", # Quan trọng để agent gốc hiểu phân công
        tools=[say_goodbye],
    )
    print(f"✅ Agent '{farewell_agent.name}' đã được tạo với model '{farewell_agent.model}'.")
except Exception as e:
    print(f"❌ Không thể tạo Farewell Agent. Kiểm tra API Key ({farewell_agent.model}). Lỗi: {e}")

✅ Agent 'farewell_agent' đã được tạo với model 'gemini-2.0-flash'.


## Định nghĩa Agent gốc (Weather Agent v2) với các Sub-Agent

In [18]:
root_agent = None
runner_root = None

if greeting_agent and farewell_agent and 'get_weather' in globals():
    # Sử dụng mô hình Gemini mạnh để điều phối
    root_agent_model = MODEL_GEMINI_2_0_FLASH

    weather_agent_team = Agent(
        name="weather_agent_v2",  # Đặt tên phiên bản mới
        model=root_agent_model,
        description="Agent điều phối chính. Xử lý yêu cầu thời tiết và ủy quyền lời chào/tạm biệt cho các agent chuyên biệt.",
        instruction="Bạn là Weather Agent chính, phụ trách điều phối một nhóm agent. Nhiệm vụ chính của bạn là cung cấp thông tin thời tiết. "
                    "Chỉ sử dụng công cụ 'get_weather' cho các yêu cầu liên quan đến thời tiết (ví dụ: 'thời tiết ở London'). "
                    "Bạn có các agent chuyên biệt như sau: "
                    "1. 'greeting_agent': Xử lý lời chào đơn giản như 'Hi', 'Hello'. Hãy ủy quyền cho nó khi gặp các tình huống này. "
                    "2. 'farewell_agent': Xử lý lời tạm biệt đơn giản như 'Bye', 'See you'. Hãy ủy quyền cho nó khi gặp các tình huống này. "
                    "Phân tích câu hỏi người dùng. Nếu là lời chào, hãy ủy quyền cho 'greeting_agent'. Nếu là lời tạm biệt, hãy ủy quyền cho 'farewell_agent'. "
                    "Nếu là yêu cầu thời tiết, bạn tự xử lý bằng 'get_weather'. "
                    "Với các trường hợp khác, hãy phản hồi phù hợp hoặc thông báo rằng bạn không thể xử lý.",
        tools=[get_weather],  # Agent gốc vẫn cần tool thời tiết để xử lý
        sub_agents=[greeting_agent, farewell_agent]
    )
    print(f"✅ Đã tạo Root Agent '{weather_agent_team.name}' sử dụng mô hình '{root_agent_model}' với các sub-agent: {[sa.name for sa in weather_agent_team.sub_agents]}")
else:
    print("❌ Không thể tạo root agent vì thiếu sub-agent hoặc hàm 'get_weather'.")
    if not greeting_agent: print(" - Thiếu Greeting Agent.")
    if not farewell_agent: print(" - Thiếu Farewell Agent.")
    if 'get_weather' not in globals(): print(" - Thiếu hàm get_weather.")


✅ Đã tạo Root Agent 'weather_agent_v2' sử dụng mô hình 'gemini-2.0-flash' với các sub-agent: ['greeting_agent', 'farewell_agent']


## Tương tác với nhóm Agent

In [20]:
import asyncio

# Kiểm tra biến agent gốc có tồn tại không trước khi định nghĩa hàm trò chuyện
root_agent_var_name = 'root_agent'

if 'weather_agent_team' in globals():
    root_agent_var_name = 'weather_agent_team'
elif 'root_agent' not in globals():
    print("⚠️ Không tìm thấy agent gốc ('root_agent' hoặc 'weather_agent_team'). Không thể định nghĩa run_team_conversation.")
    root_agent = None  # Gán giá trị tạm để tránh lỗi

# Chỉ định nghĩa và chạy nếu agent gốc tồn tại
if root_agent_var_name in globals() and globals()[root_agent_var_name]:
    # Định nghĩa hàm async chính cho logic trò chuyện
    async def run_team_conversation():
        print("\n--- Đang kiểm tra tính năng phân quyền của Đội ngũ Agent ---")
        session_service = InMemorySessionService()
        APP_NAME = "weather_tutorial_agent_team"
        USER_ID = "user_1_agent_team"
        SESSION_ID = "session_001_agent_team"

        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID
        )
        print(f"Đã tạo session: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

        actual_root_agent = globals()[root_agent_var_name]
        runner_agent_team = Runner(
            agent=actual_root_agent,
            app_name=APP_NAME,
            session_service=session_service
        )
        print(f"Đã tạo runner cho agent '{actual_root_agent.name}'.")

        # --- Tương tác (dùng await trong async def) ---
        await call_agent_async(query = "Xin chào!",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)
        await call_agent_async(query = "Thời tiết ở New York như nào?",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)
        await call_agent_async(query = "Cảm ơn, tạm biệt nhé!",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)

    # --- Thực thi hàm async `run_team_conversation` ---
    print("Đang thử thực thi bằng 'await' (mặc định cho notebooks)...")
    await run_team_conversation()

Đang thử thực thi bằng 'await' (mặc định cho notebooks)...

--- Đang kiểm tra tính năng phân quyền của Đội ngũ Agent ---
Đã tạo session: App='weather_tutorial_agent_team', User='user_1_agent_team', Session='session_001_agent_team'
Đã tạo runner cho agent 'weather_agent_v2'.

>>> Câu hỏi từ người dùng: Xin chào!


  [Event] Tác giả: weather_agent_v2, Loại: Event, Cuối cùng: False, Nội dung: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=FunctionCall(id='adk-bf991169-77b4-4761-84ae-5467371620f1', args={'agent_name': 'greeting_agent'}, name='transfer_to_agent'), function_response=None, text=None)] role='model'
  [Event] Tác giả: weather_agent_v2, Loại: Event, Cuối cùng: False, Nội dung: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=FunctionResponse(will_continue=None, scheduling=None, id='adk-bf991169-77b4-4761-84ae-5467371620f1', name='transfer_to_agent', response={'result': None}), text=None)] role='user'


  [Event] Tác giả: greeting_agent, Loại: Event, Cuối cùng: False, Nội dung: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=FunctionCall(id='adk-2da88055-4256-4526-b0eb-9ed5eff117c8', args={}, name='say_hello'), function_response=None, text=None)] role='model'
--- Tool: say_hello được gọi mà không có tên cụ thể (giá trị name: None) ---
  [Event] Tác giả: greeting_agent, Loại: Event, Cuối cùng: False, Nội dung: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=FunctionResponse(will_continue=None, scheduling=None, id='adk-2da88055-4256-4526-b0eb-9ed5eff117c8', name='say_hello', response={'result': 'Xin chào!'}), text=None)] role='user'
  [Event] Tác giả: greeting_agent, Loại: Event, Cuối cùng: True, Nội dung: parts=[Part(video_m

  [Event] Tác giả: greeting_agent, Loại: Event, Cuối cùng: False, Nội dung: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=FunctionCall(id='adk-2cb53e8f-be4f-44c3-b942-d9828dd9b058', args={'agent_name': 'weather_agent_v2'}, name='transfer_to_agent'), function_response=None, text=None)] role='model'
  [Event] Tác giả: greeting_agent, Loại: Event, Cuối cùng: False, Nội dung: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=FunctionResponse(will_continue=None, scheduling=None, id='adk-2cb53e8f-be4f-44c3-b942-d9828dd9b058', name='transfer_to_agent', response={'result': None}), text=None)] role='user'


  [Event] Tác giả: weather_agent_v2, Loại: Event, Cuối cùng: False, Nội dung: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=FunctionCall(id='adk-e5ebec88-488e-45a9-97aa-e55903266064', args={'city': 'New York'}, name='get_weather'), function_response=None, text=None)] role='model'
--- Tool: get_weather được gọi cho thành phố: New York ---
  [Event] Tác giả: weather_agent_v2, Loại: Event, Cuối cùng: False, Nội dung: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=FunctionResponse(will_continue=None, scheduling=None, id='adk-e5ebec88-488e-45a9-97aa-e55903266064', name='get_weather', response={'status': 'success', 'report': 'Thời tiết ở New York nắng, nhiệt độ 25°C.'}), text=None)] role='user'
  [Event] Tác giả: weather_agent_

  [Event] Tác giả: weather_agent_v2, Loại: Event, Cuối cùng: False, Nội dung: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=FunctionCall(id='adk-0fc945da-2d94-4ea6-a24c-e6aee8509ccd', args={'agent_name': 'farewell_agent'}, name='transfer_to_agent'), function_response=None, text=None)] role='model'
  [Event] Tác giả: weather_agent_v2, Loại: Event, Cuối cùng: False, Nội dung: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=FunctionResponse(will_continue=None, scheduling=None, id='adk-0fc945da-2d94-4ea6-a24c-e6aee8509ccd', name='transfer_to_agent', response={'result': None}), text=None)] role='user'


  [Event] Tác giả: farewell_agent, Loại: Event, Cuối cùng: False, Nội dung: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=FunctionCall(id='adk-444f4dc0-9126-4f71-8600-15b5968eca47', args={}, name='say_goodbye'), function_response=None, text=None)] role='model'
--- Tool: say_goodbye được gọi ---
  [Event] Tác giả: farewell_agent, Loại: Event, Cuối cùng: False, Nội dung: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=FunctionResponse(will_continue=None, scheduling=None, id='adk-444f4dc0-9126-4f71-8600-15b5968eca47', name='say_goodbye', response={'result': 'Tạm biệt! Chúc bạn một ngày tốt lành.'}), text=None)] role='user'
  [Event] Tác giả: farewell_agent, Loại: Event, Cuối cùng: True, Nội dung: parts=[Part(video_metadata=No

# Step 4: Thêm ghi nhớ và cá nhân hóa với session state

In [21]:
# Nhập các thành phần cần thiết để quản lý phiên làm việc
from google.adk.sessions import InMemorySessionService

# Tạo một instance mới của InMemorySessionService để thử nghiệm trạng thái
session_service_stateful = InMemorySessionService()
print("✅ Đã tạo InMemorySessionService mới cho phần trình diễn trạng thái.")

# Đặt ID mới cho session
SESSION_ID_STATEFUL = "session_state_demo_001"
USER_ID_STATEFUL = "user_state_demo"

# Khai báo trạng thái ban đầu – người dùng muốn dùng đơn vị Celsius
initial_state = {
    "user_preference_temperature_unit": "Celsius"
}

# Tạo session mới và cung cấp trạng thái ban đầu
session_stateful = await session_service_stateful.create_session(
    app_name=APP_NAME,  # Giữ nguyên tên ứng dụng
    user_id=USER_ID_STATEFUL,
    session_id=SESSION_ID_STATEFUL,
    state=initial_state  # <<< Khởi tạo session.state tại đây
)
print(f"✅ Session '{SESSION_ID_STATEFUL}' đã được tạo cho người dùng '{USER_ID_STATEFUL}'.")

# Kiểm tra lại session vừa tạo và in ra trạng thái ban đầu
retrieved_session = await session_service_stateful.get_session(app_name=APP_NAME,
                                                         user_id=USER_ID_STATEFUL,
                                                         session_id = SESSION_ID_STATEFUL)
print("\n--- Trạng thái session ban đầu ---")
if retrieved_session:
    print(retrieved_session.state)
else:
    print("Lỗi: Không thể truy xuất session.")


✅ Đã tạo InMemorySessionService mới cho phần trình diễn trạng thái.
✅ Session 'session_state_demo_001' đã được tạo cho người dùng 'user_state_demo'.

--- Trạng thái session ban đầu ---
{'user_preference_temperature_unit': 'Celsius'}


In [23]:
from google.adk.tools.tool_context import ToolContext

def get_weather_stateful(city: str, tool_context: ToolContext) -> dict:
    """Lấy thông tin thời tiết và định dạng đơn vị nhiệt độ dựa theo trạng thái phiên làm việc của người dùng."""
    print(f"--- Tool: get_weather_stateful được gọi với thành phố {city} ---")

    # --- Đọc đơn vị nhiệt độ ưa thích từ session state ---
    preferred_unit = tool_context.state.get("user_preference_temperature_unit", "Celsius")  # Mặc định là Celsius nếu không có
    print(f"--- Tool: Đọc state 'user_preference_temperature_unit': {preferred_unit} ---")

    city_normalized = city.lower().replace(" ", "")  # Chuẩn hóa tên thành phố (thường dùng để tra key)

    # Cơ sở dữ liệu mô phỏng thời tiết (nhiệt độ luôn lưu theo Celsius)
    mock_weather_db = {
        "newyork": {"temp_c": 25, "condition": "sunny"},
        "london": {"temp_c": 15, "condition": "cloudy"},
        "tokyo": {"temp_c": 18, "condition": "light rain"},
    }

    if city_normalized in mock_weather_db:
        data = mock_weather_db[city_normalized]
        temp_c = data["temp_c"]
        condition = data["condition"]

        # Chuyển đổi đơn vị nhiệt độ nếu cần
        if preferred_unit == "Fahrenheit":
            temp_value = (temp_c * 9/5) + 32  # Đổi sang Fahrenheit
            temp_unit = "°F"
        else:
            temp_value = temp_c
            temp_unit = "°C"

        # Tạo bản báo cáo thời tiết
        report = f"The weather in {city.capitalize()} is {condition} with a temperature of {temp_value:.0f}{temp_unit}."
        result = {"status": "success", "report": report}
        print(f"--- Tool: Tạo báo cáo thời tiết theo {preferred_unit}. Kết quả: {result} ---")

        # (Tùy chọn) Ghi lại tên thành phố vừa được tra vào state
        tool_context.state["last_city_checked_stateful"] = city
        print(f"--- Tool: Cập nhật state 'last_city_checked_stateful': {city} ---")

        return result
    else:
        # Trường hợp không tìm thấy thành phố
        error_msg = f"Sorry, I don't have weather information for '{city}'."
        print(f"--- Tool: Thành phố '{city}' không có trong dữ liệu. ---")
        return {"status": "error", "error_message": error_msg}

print("✅ Đã định nghĩa công cụ 'get_weather_stateful' có hỗ trợ trạng thái.")

✅ Đã định nghĩa công cụ 'get_weather_stateful' có hỗ trợ trạng thái.


In [25]:
# Định nghĩa lại các Agent con và cập nhật Root Agent có hỗ trợ trạng thái (state)

from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm
from google.adk.runners import Runner

# --- Định nghĩa lại Greeting Agent ---
greeting_agent = None
try:
    greeting_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="greeting_agent",
        instruction="Bạn là Greeting Agent. Nhiệm vụ DUY NHẤT của bạn là chào hỏi người dùng bằng cách dùng công cụ 'say_hello'. Không làm gì khác.",
        description="Xử lý lời chào đơn giản bằng công cụ 'say_hello'.",
        tools=[say_hello],
    )
    print(f"✅ Đã định nghĩa lại agent '{greeting_agent.name}'.")
except Exception as e:
    print(f"❌ Không thể định nghĩa Greeting agent. Lỗi: {e}")


# --- Định nghĩa lại Farewell Agent ---
farewell_agent = None
try:
    farewell_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="farewell_agent",
        instruction="Bạn là Farewell Agent. Nhiệm vụ DUY NHẤT là chào tạm biệt người dùng bằng công cụ 'say_goodbye'. Không thực hiện tác vụ khác.",
        description="Xử lý lời tạm biệt đơn giản bằng công cụ 'say_goodbye'.",
        tools=[say_goodbye],
    )
    print(f"✅ Đã định nghĩa lại agent '{farewell_agent.name}'.")
except Exception as e:
    print(f"❌ Không thể định nghĩa Farewell agent. Lỗi: {e}")


# --- Tạo Root Agent mới có hỗ trợ trạng thái ---
root_agent_stateful = None
runner_root_stateful = None  # Tạo biến runner

# Kiểm tra các thành phần cần thiết đã có chưa
if greeting_agent and farewell_agent and 'get_weather_stateful' in globals():

    root_agent_model = MODEL_GEMINI_2_0_FLASH

    root_agent_stateful = Agent(
        name="weather_agent_v4_stateful",  # Tên agent mới (v4, có state)
        model=root_agent_model,
        description="Agent chính: Cung cấp thời tiết (có xét trạng thái người dùng), chuyển lời chào/tạm biệt cho agent con, và lưu kết quả thời tiết vào state.",
        instruction="Bạn là Weather Agent chính. Công việc của bạn là cung cấp thông tin thời tiết bằng công cụ 'get_weather_stateful'. "
                    "Công cụ này sẽ định dạng nhiệt độ dựa theo sở thích của người dùng lưu trong state. "
                    "Hãy chuyển tiếp các câu chào đơn giản cho 'greeting_agent' và tạm biệt cho 'farewell_agent'. "
                    "Chỉ xử lý các yêu cầu về thời tiết, lời chào và lời tạm biệt.",
        tools=[get_weather_stateful],  # Sử dụng công cụ mới hỗ trợ đọc state
        sub_agents=[greeting_agent, farewell_agent],  # Thêm các agent con
        output_key="last_weather_report"  # Dòng này giúp lưu phản hồi cuối cùng vào session.state["last_weather_report"]
    )
    print(f"✅ Đã tạo Root Agent '{root_agent_stateful.name}' có hỗ trợ state và output_key.")

    # --- Tạo Runner để chạy Root Agent này ---
    runner_root_stateful = Runner(
        agent=root_agent_stateful,
        app_name=APP_NAME,  # Tên ứng dụng bạn đã định nghĩa từ trước
        session_service=session_service_stateful  # Dùng dịch vụ session có hỗ trợ state
    )
    print(f"✅ Đã tạo Runner cho Root Agent có state '{runner_root_stateful.agent.name}'.")

else:
    print("❌ Không thể tạo Root Agent có state. Thiếu các thành phần cần thiết.")
    if not greeting_agent:
        print(" - Thiếu định nghĩa greeting_agent.")
    if not farewell_agent:
        print(" - Thiếu định nghĩa farewell_agent.")
    if 'get_weather_stateful' not in globals():
        print(" - Thiếu tool get_weather_stateful.")


✅ Đã định nghĩa lại agent 'greeting_agent'.
✅ Đã định nghĩa lại agent 'farewell_agent'.
✅ Đã tạo Root Agent 'weather_agent_v4_stateful' có hỗ trợ state và output_key.
✅ Đã tạo Runner cho Root Agent có state 'weather_agent_v4_stateful'.


In [26]:
if 'runner_root_stateful' in globals() and runner_root_stateful:

    # Định nghĩa hàm async chính để kiểm tra hội thoại có sử dụng state
    async def run_stateful_conversation():
        print("\n--- Đang kiểm tra state: Đổi đơn vị nhiệt độ & output_key ---")

        # 1. Gọi kiểm tra thời tiết với đơn vị mặc định (Celsius)
        print("--- Bước 1: Yêu cầu thời tiết ở London (kỳ vọng là Celsius) ---")
        await call_agent_async(query="What's the weather in London?",
                               runner=runner_root_stateful,
                               user_id=USER_ID_STATEFUL,
                               session_id=SESSION_ID_STATEFUL)

        # 2. Cập nhật trực tiếp trạng thái: Đổi đơn vị sang Fahrenheit
        print("\n--- Cập nhật thủ công state: Chuyển đơn vị sang Fahrenheit ---")
        try:
            # Truy cập trực tiếp vào lưu trữ nội bộ - chỉ áp dụng cho InMemorySessionService (dùng để test)
            stored_session = session_service_stateful.sessions[APP_NAME][USER_ID_STATEFUL][SESSION_ID_STATEFUL]
            stored_session.state["user_preference_temperature_unit"] = "Fahrenheit"
            print(f"--- Trạng thái đã cập nhật. Giá trị hiện tại: {stored_session.state.get('user_preference_temperature_unit', 'Không có')} ---")
        except KeyError:
            print(f"--- Lỗi: Không thể lấy phiên làm việc '{SESSION_ID_STATEFUL}' để cập nhật ---")
        except Exception as e:
            print(f"--- Lỗi khi cập nhật session state: {e} ---")

        # 3. Gọi lại kiểm tra thời tiết với state đã cập nhật (kỳ vọng Fahrenheit)
        print("\n--- Bước 2: Yêu cầu thời tiết ở New York (kỳ vọng là Fahrenheit) ---")
        await call_agent_async(query="Tell me the weather in New York.",
                               runner=runner_root_stateful,
                               user_id=USER_ID_STATEFUL,
                               session_id=SESSION_ID_STATEFUL)

        # 4. Kiểm tra agent con hoạt động song song với state (chào hỏi)
        print("\n--- Bước 3: Gửi lời chào ---")
        await call_agent_async(query="Hi!",
                               runner=runner_root_stateful,
                               user_id=USER_ID_STATEFUL,
                               session_id=SESSION_ID_STATEFUL)

    # Thực thi hàm async (chỉ dùng được trong môi trường hỗ trợ top-level await như notebook)
    print("Đang thực thi bằng 'await' (mặc định cho notebook)...")
    await run_stateful_conversation()

    # --- Kiểm tra lại trạng thái cuối sau hội thoại ---
    print("\n--- Kiểm tra trạng thái phiên làm việc cuối cùng ---")
    final_session = await session_service_stateful.get_session(app_name=APP_NAME,
                                                               user_id=USER_ID_STATEFUL,
                                                               session_id=SESSION_ID_STATEFUL)
    if final_session:
        # In từng thành phần trong state
        print(f"Đơn vị nhiệt độ hiện tại: {final_session.state.get('user_preference_temperature_unit', 'Không có')}")
        print(f"Báo cáo thời tiết cuối (từ output_key): {final_session.state.get('last_weather_report', 'Không có')}")
        print(f"Thành phố kiểm tra cuối cùng (do công cụ ghi): {final_session.state.get('last_city_checked_stateful', 'Không có')}")
    else:
        print("\n❌ Lỗi: Không thể lấy session state cuối cùng.")

else:
    print("\n⚠️ Bỏ qua: Runner agent có state ('runner_root_stateful') không tồn tại.")


Đang thực thi bằng 'await' (mặc định cho notebook)...

--- Đang kiểm tra state: Đổi đơn vị nhiệt độ & output_key ---
--- Bước 1: Yêu cầu thời tiết ở London (kỳ vọng là Celsius) ---

>>> Câu hỏi từ người dùng: What's the weather in London?


  [Event] Tác giả: weather_agent_v4_stateful, Loại: Event, Cuối cùng: False, Nội dung: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=FunctionCall(id='adk-9025ac59-e1d5-411e-99d9-6b770442f5e9', args={'city': 'London'}, name='get_weather_stateful'), function_response=None, text=None)] role='model'
--- Tool: get_weather_stateful được gọi với thành phố London ---
--- Tool: Đọc state 'user_preference_temperature_unit': Celsius ---
--- Tool: Tạo báo cáo thời tiết theo Celsius. Kết quả: {'status': 'success', 'report': 'The weather in London is cloudy with a temperature of 15°C.'} ---
--- Tool: Cập nhật state 'last_city_checked_stateful': London ---
  [Event] Tác giả: weather_agent_v4_stateful, Loại: Event, Cuối cùng: False, Nội dung: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executa

  [Event] Tác giả: weather_agent_v4_stateful, Loại: Event, Cuối cùng: False, Nội dung: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=FunctionCall(id='adk-8503e9e7-672a-4bb4-b70d-12dbff85dea3', args={'city': 'New York'}, name='get_weather_stateful'), function_response=None, text=None)] role='model'
--- Tool: get_weather_stateful được gọi với thành phố New York ---
--- Tool: Đọc state 'user_preference_temperature_unit': Fahrenheit ---
--- Tool: Tạo báo cáo thời tiết theo Fahrenheit. Kết quả: {'status': 'success', 'report': 'The weather in New york is sunny with a temperature of 77°F.'} ---
--- Tool: Cập nhật state 'last_city_checked_stateful': New York ---
  [Event] Tác giả: weather_agent_v4_stateful, Loại: Event, Cuối cùng: False, Nội dung: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=

  [Event] Tác giả: weather_agent_v4_stateful, Loại: Event, Cuối cùng: False, Nội dung: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=FunctionCall(id='adk-fbd5f214-c092-4dc1-81aa-eeddef5dc65b', args={'agent_name': 'greeting_agent'}, name='transfer_to_agent'), function_response=None, text=None)] role='model'
  [Event] Tác giả: weather_agent_v4_stateful, Loại: Event, Cuối cùng: False, Nội dung: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=FunctionResponse(will_continue=None, scheduling=None, id='adk-fbd5f214-c092-4dc1-81aa-eeddef5dc65b', name='transfer_to_agent', response={'result': None}), text=None)] role='user'


  [Event] Tác giả: greeting_agent, Loại: Event, Cuối cùng: False, Nội dung: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=FunctionCall(id='adk-07fde593-824b-4849-8b0e-a0ffa2b15e1e', args={}, name='say_hello'), function_response=None, text=None)] role='model'
--- Tool: say_hello được gọi mà không có tên cụ thể (giá trị name: None) ---
  [Event] Tác giả: greeting_agent, Loại: Event, Cuối cùng: False, Nội dung: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=FunctionResponse(will_continue=None, scheduling=None, id='adk-07fde593-824b-4849-8b0e-a0ffa2b15e1e', name='say_hello', response={'result': 'Xin chào!'}), text=None)] role='user'
  [Event] Tác giả: greeting_agent, Loại: Event, Cuối cùng: True, Nội dung: parts=[Part(video_m